In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from time import strftime, localtime

# === CONFIGURACIÓN ===
video_path = "video_celular.mp4"  # Cambiá este nombre por el de tu archivo
fps = 30  # Cambiá esto si tu video tiene otro FPS

# === Crear carpeta del experimento ===
timestamp_str = strftime("experimento_%Y%m%d_%H%M%S", localtime())
os.makedirs(timestamp_str, exist_ok=True)

# === Cargar video ===
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError(f"No se pudo abrir el video: {video_path}")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# === Guardar videos ===
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_original = cv2.VideoWriter(f"{timestamp_str}/video_original.avi", fourcc, fps, (width, height))
out_red = cv2.VideoWriter(f"{timestamp_str}/video_rojo.avi", fourcc, fps, (width, height), False)
out_blue = cv2.VideoWriter(f"{timestamp_str}/video_azul.avi", fourcc, fps, (width, height), False)

# === Variables ===
datos = []
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    timestamp = frame_count / fps
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # --- Filtro rojo (dos rangos HSV) ---
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    mask_red = cv2.inRange(hsv, lower_red1, upper_red1) | cv2.inRange(hsv, lower_red2, upper_red2)

    # --- Filtro azul oscuro ---
    lower_blue = np.array([100, 150, 50])
    upper_blue = np.array([130, 255, 255])
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # --- Centro de masa rojo ---
    M_red = cv2.moments(mask_red)
    if M_red["m00"] != 0:
        cx_red = int(M_red["m10"] / M_red["m00"])
        cy_red = int(M_red["m01"] / M_red["m00"])
    else:
        cx_red, cy_red = np.nan, np.nan

    # --- Centro de masa azul ---
    M_blue = cv2.moments(mask_blue)
    if M_blue["m00"] != 0:
        cx_blue = int(M_blue["m10"] / M_blue["m00"])
        cy_blue = int(M_blue["m01"] / M_blue["m00"])
    else:
        cx_blue, cy_blue = np.nan, np.nan

    # === Guardar datos ===
    datos.append([timestamp, cx_red, cy_red, cx_blue, cy_blue])

    # === Mostrar frame original sin marcas ===
    cv2.imshow("Vista original", frame)

    # === Guardar frames sin marcas ===
    out_original.write(frame)
    out_red.write(mask_red)
    out_blue.write(mask_blue)

    # === Mostrar gráfica cada 10 s ===
    if int(timestamp) % 10 == 0 and frame_count != 0 and frame_count % (10 * fps) == 0:
        df = pd.DataFrame(datos, columns=["tiempo_s", "x_rojo", "y_rojo", "x_azul", "y_azul"])
        plt.figure(figsize=(10, 6))
        plt.subplot(2, 1, 1)
        plt.plot(df["tiempo_s"], df["x_rojo"], 'ro', label="x rojo")
        plt.plot(df["tiempo_s"], df["x_azul"], 'bo', label="x azul")
        plt.ylabel("X (px)")
        plt.legend()
        plt.subplot(2, 1, 2)
        plt.plot(df["tiempo_s"], df["y_rojo"], 'r^', label="y rojo")
        plt.plot(df["tiempo_s"], df["y_azul"], 'b^', label="y azul")
        plt.ylabel("Y (px)")
        plt.xlabel("Tiempo (s)")
        plt.legend()
        plt.tight_layout()
        plt.show()

    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# === Guardar CSV final ===
df_final = pd.DataFrame(datos, columns=["tiempo_s", "x_rojo", "y_rojo", "x_azul", "y_azul"])
df_final.to_csv(f"{timestamp_str}/trayectorias.csv", index=False)

# === Liberar ===
cap.release()
out_original.release()
out_red.release()
out_blue.release()
cv2.destroyAllWindows()
print(f"✅ Datos guardados en la carpeta: {timestamp_str}")